In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
df = pd.read_csv('preprocessed_DNN (5).csv', low_memory=False)
df

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.response,http.tls_port,tcp.ack,tcp.ack_raw,...,dns.qry.name.len-1.debian.pool.ntp.org,dns.qry.name.len-2.debian.pool.ntp.org,dns.qry.name.len-3.debian.pool.ntp.org,dns.qry.name.len-_googlecast._tcp.local,mqtt.conack.flags-0,mqtt.conack.flags-0x00000000,mqtt.protoname-0,mqtt.protoname-MQTT,mqtt.topic-0,mqtt.topic-Temperature_and_Humidity
0,0,0,0,0,0,0,0,0,110218025,1249069932,...,False,False,False,False,True,False,True,False,True,False
1,0,0,0,0,0,0,0,0,1,3306662301,...,False,False,False,False,True,False,True,False,True,False
2,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,True,False,True,False,True,False
3,0,0,0,0,0,0,0,0,930,142931531,...,False,False,False,False,True,False,True,False,True,False
4,0,0,0,0,0,0,0,0,442,984983596,...,False,False,False,False,True,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152191,0,0,0,0,0,0,0,0,1,2013966124,...,False,False,False,False,True,False,True,False,True,False
152192,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,True,False,True,False,True,False
152193,0,0,0,0,0,0,0,0,103,1752254882,...,False,False,False,False,True,False,True,False,True,False
152194,0,0,0,0,0,0,0,0,719652,2372344333,...,False,False,False,False,True,False,True,False,True,False


In [5]:
feat_cols = list(df.columns)
label_col = "Attack_type"

feat_cols.remove(label_col)

In [6]:
skip_list = ["icmp.unused", "http.tls_port", "dns.qry.type", "mqtt.msg_decoded_as"]

In [7]:
df[skip_list[3]].value_counts()

mqtt.msg_decoded_as
0    152196
Name: count, dtype: int64

In [8]:
X = df.drop([label_col], axis=1)
y = df[label_col]

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

del X
del y

In [10]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
y_train =  label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [11]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X_train =  min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

In [12]:
#X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
#X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [13]:
print(X_train.shape)
print(X_test.shape)

(121756, 68)
(30440, 68)


In [14]:
input_shape = X_train.shape[1:]

In [15]:
print(X_train.shape, X_test.shape)
print(input_shape)

(121756, 68) (30440, 68)
(68,)


In [16]:
num_classes = len(np.unique(y_train))
num_classes

15

In [17]:
from  tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

2025-07-04 13:54:09.174362: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 13:54:09.175890: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-04 13:54:09.194643: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-04 13:54:09.194675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-04 13:54:09.195245: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [18]:
print(y_train.shape, y_test.shape)

(121756, 15) (30440, 15)


In [19]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, MultiHeadAttention,
    LayerNormalization, Add, Dropout, Dense, GlobalAveragePooling1D, Reshape
)

# Data shape:
num_features = 68
num_classes = 15

# For Conv1D, reshape input to (sequence_length, num_channels)
# Here, treat features as "sequence_length = 68", with 1 channel per feature
inputs = Input(shape=(num_features, 1))  # reshape your data to (batch, 68, 1)

# CNN feature extractor
x = Conv1D(32, 3, activation='relu', padding='same')(inputs)
x = MaxPooling1D(2)(x)

x = Conv1D(64, 3, activation='relu', padding='same')(x)
x = MaxPooling1D(2)(x)

# Transformer block
attention_output = MultiHeadAttention(num_heads=4, key_dim=16)(x, x)
attention_output = Dropout(0.1)(attention_output)
x = Add()([x, attention_output])
x = LayerNormalization(epsilon=1e-6)(x)

ffn = tf.keras.Sequential([
    Dense(128, activation='relu'),
    Dense(x.shape[-1])
])
ffn_output = ffn(x)
x = Add()([x, ffn_output])
x = LayerNormalization(epsilon=1e-6)(x)

# Classification head
x = GlobalAveragePooling1D()(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 68, 1)]              0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 68, 32)               128       ['input_1[0][0]']             
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 34, 32)               0         ['conv1d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv1d_1 (Conv1D)           (None, 34, 64)               6208      ['max_pooling1d[0][0]']   

In [20]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # or 'sparse_categorical_crossentropy'
    metrics=['accuracy']
)

In [21]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from livelossplot import PlotLossesKeras


#model_weights_file_path = "model_best_weights.h5"
#checkpoint = ModelCheckpoint(filepath=model_weights_file_path, monitor="val_loss", verbose=1, save_best_only=True, mode="min", save_weights_only=True)
#checkpoint = ModelCheckpoint(filepath=model_weights_file_path, monitor="val_loss", verbose=1, save_best_only=True, mode="min", save_weights_only=True)
early_stopping = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, mode="min", verbose=1, min_lr=0)
#plotlosses = PlotLossesKeras()
call_backs = [ early_stopping, lr_reduce]
EPOCHS = 15
BATCH_SIZE = 256
call_backs = [early_stopping, lr_reduce]
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    validation_split=0.1,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    callbacks=call_backs,
                    #class_weight=class_weights,
                    verbose=1)

Epoch 1/15
476/476 [==============================] - 6s 10ms/step - loss: 1.4479 - accuracy: 0.4796 - val_loss: 1.0959 - val_accuracy: 0.5630 - lr: 0.0010
Epoch 2/15
476/476 [==============================] - 5s 10ms/step - loss: 1.0629 - accuracy: 0.5671 - val_loss: 1.0317 - val_accuracy: 0.5694 - lr: 0.0010
Epoch 3/15
476/476 [==============================] - 5s 10ms/step - loss: 1.0044 - accuracy: 0.5884 - val_loss: 0.9822 - val_accuracy: 0.5933 - lr: 0.0010
Epoch 4/15
476/476 [==============================] - 5s 10ms/step - loss: 0.9717 - accuracy: 0.5992 - val_loss: 0.9543 - val_accuracy: 0.6017 - lr: 0.0010
Epoch 5/15
476/476 [==============================] - 5s 10ms/step - loss: 0.9462 - accuracy: 0.6084 - val_loss: 0.9164 - val_accuracy: 0.6270 - lr: 0.0010
Epoch 6/15
476/476 [==============================] - 5s 10ms/step - loss: 0.9224 - accuracy: 0.6206 - val_loss: 0.9002 - val_accuracy: 0.6308 - lr: 0.0010
Epoch 7/15
476/476 [==============================] - 5s 10ms/st